In [3]:
import os
import tarfile
import urllib.request as url
import numpy as np

# download cifar
check if CIFAR in directory, if not, download 

In [4]:
def download_data():
    if not os.path.exists('.././data'):
        os.mkdir('.././data')
    if not os.path.exists('.././data/cifar-10-python.tar.gz'):
        print("CIFAR-10 not found, downloading...")
        url.urlretrieve("https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz",
                        ".././data/cifar-10-python.tar.gz")
        print("done")
        print("Download completed!")
    else:
        print('CIFAR-10 already exists.')
        
# download_data()

In [5]:
import glob
import _pickle as pickle
def load_data():
    if not os.path.exists('.././data/cifar-10-python.tar.gz'):
        download_data()
    if not os.path.exists('.././data/cifar-10-batches-py/'):
        print("extracting...")
        package = tarfile.open('.././data/cifar-10-python.tar.gz')
        package.extractall('.././data')
        package.close()
        
    root_dir = os.getcwd()
    os.chdir('.././data/cifar-10-batches-py')
    train_data = []
    train_label = []
    test_data = []
    test_label = []
    data_train = glob.glob('data_batch*')
    print(data_train)
    #try:
    for name in data_train:
        handle = open(name, 'rb')
        cmap = pickle.load(handle, encoding='bytes')
        train_data.append(cmap[b'data'])
        train_label.append(cmap[b'labels'])
        handle.close()
    # Turn the dataset into numpy compatible arrays.
    train_data = np.concatenate(train_data, axis=0)
    train_label = np.concatenate(train_label)
    handle = open('test_batch', 'rb')
    cmap = pickle.load(handle, encoding='bytes')
    test_data.append(cmap[b'data'])
    test_label.append(cmap[b'labels'])
    test_data = np.array(test_data[0])
    test_label = np.array(test_label[0])
    #except BaseException:
#         os.chdir(root_dir)
#         print('Something went wrong...')
#         return None
    os.chdir(root_dir)
    return train_data,train_label,test_data,test_label

train_data,train_label,test_data,test_label = load_data()
val_size = 1000
val_data = train_data[-1000:]
val_label = train_label[-1000:]
train_data = train_data[:-1000]
train_label = train_label [:-1000]
print(train_data.shape, train_label.shape)
print(val_data.shape, val_label.shape)
print(test_data.shape, test_label.shape)

['data_batch_1', 'data_batch_4', 'data_batch_3', 'data_batch_2', 'data_batch_5']
(49000, 3072) (49000,)
(1000, 3072) (1000,)
(10000, 3072) (10000,)


# build neural network using tensorflow


In [6]:
import tensorflow as tf
def cnn_4_cifar(features, labels, mode = tf.estimator.ModeKeys.TRAIN):
    # features aka. x, labels aka y, i don't know mode right now
    train_x = features
    train_y = labels
    input_data = tf.placeholder(
        dtype = tf.float32,
        shape = [None, 3072],
        name = "input")
    
    labels = tf.placeholder(
        dtype = tf.int32,
        shape = [None, ],
        name = "labels")
    
    input_labels = tf.one_hot(indices=labels,
                        depth = 10,
                        name='one_hot_label',
                        axis=1,
                       dtype = tf.int32)
    
    ###### input layer ######
    
    input_layer = tf.reshape(input_data, [-1, 32, 32, 3])
    input_layer = tf.cast(input_layer, dtype = tf.float32)
    
    ###### normalize layer ######
    
    normal_layer = tf.layers.batch_normalization(
        inputs = input_layer,
        training= mode == tf.estimator.ModeKeys.TRAIN,
        reuse=None,
        name = 'batch_normal'
    )
    #normalize_layer = tf.nn.l2_normalize()
    
    ###### convolutional layer #1 ######
    
    conv1 = tf.layers.conv2d(
        inputs = normal_layer,
        filters = 18,
        kernel_size = [3,3],
        padding = "same",
        activation = tf.nn.relu
    )
    
    ###### pooling layer #1 ######
    
    pool1 = tf.layers.max_pooling2d(
        inputs=conv1, pool_size = [2,2], strides = 2)
    
    
    ###### convolutional layer #2 ######
        
    conv2 = tf.layers.conv2d(
        inputs = pool1,
        filters = 64,
        kernel_size = [3,3],
        padding = "same",
        activation = tf.nn.relu
    )
    
    ###### pooling layer #1 ######
    
    pool2 = tf.layers.max_pooling2d(
        inputs = conv2,
        pool_size = [2,2],
        strides = 2)
    
    ###### flat layer ######
    
    pool_flat = tf.reshape(pool2, [-1, 8 * 8 * 64])
    
    ###### dense layer ######
    
    dense1 = tf.layers.dense(
        inputs = pool_flat,                     
        units = 1024,                     
        activation = tf.nn.relu
    )
    
    dense2 = tf.layers.dense(
        inputs = dense1,                     
        units = 200,                     
        activation = tf.nn.relu
    )
    
    ###### dropout layer ######
    
    dropout = tf.layers.dropout(
        inputs = dense2,
        rate = 0.2,
        training = (mode == tf.estimator.ModeKeys.TRAIN)
    )
    ###### logits layer ######
    
    logits = tf.layers.dense(inputs=dropout, units=10)
    
    ###### prediction ######
    
    predictions = {
        "class" : tf.argmax(input = logits, axis = 1),
        "prob" : tf.nn.softmax(
            logits, 
            name = "softmax_tensor")
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(
            mode=mode, 
            predictions = predictions)
    
    ###### loss ######
    
    loss = tf.losses.sparse_softmax_cross_entropy(
        labels = labels,
        logits = logits
    )
    
    ###### train op ######
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(
            learning_rate = 0.001
        )
        train_op = optimizer.minimize(
            loss = loss,
            global_step = tf.train.get_global_step()
        )
        return tf.estimator.EstimatorSpec(
            mode = mode,
            loss = loss,
            train_op = train_op
            )
    
    ##### eval op ######
    if mode == tf.estimator.ModeKeys.EVAL:
        eval_metric_ops = {
            "accuracy": tf.metrics.accuracy(
                labels=labels,
                predictions=predictions["classes"])}
    return tf.estimator.EstimatorSpec(
        mode=mode,
        loss=loss,
        eval_metric_ops=eval_metric_ops)
    
    config = tf.ConfigProto()
    config.gpu_options.allow_growth=True
    sess = tf.Session(config=config)
    sess.run(tf.global_variables_initializer())
    return sess.run(logits, feed_dict =
                    {input_data: train_x,
                    labels: train_y})



print(cnn_4_cifar(train_data[0:2], train_label[0:2]))

# config = tf.ConfigProto()
# config.gpu_options.allow_growth=True
# sess = tf.Session(config=config)

EstimatorSpec(predictions={}, loss=<tf.Tensor 'sparse_softmax_cross_entropy_loss/value:0' shape=() dtype=float32>, train_op=<tf.Operation 'GradientDescent' type=NoOp>, eval_metric_ops={}, export_outputs=None, training_chief_hooks=(), training_hooks=(), scaffold=<tensorflow.python.training.monitored_session.Scaffold object at 0x7f0c21e972b0>)


In [15]:
###### estimator ######

cifar_classifier = tf.estimator.Estimator(model_fn = cnn_4_cifar, model_dir="/tmp/mnist_convnet_model")

###### log ######

tensors_to_log = {"probabilities": "softmax_tensor"}
logging_hook = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=50)

###### training ######
tf.reshape(train_data, [-1, 3072])

train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"input": train_data},
    y=train_label,
    batch_size=100,
    num_epochs=None,
    shuffle=True)
cifar_classifier.train(
    input_fn=train_input_fn,
    steps=20000,
    hooks=[logging_hook])






INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': 1, '_save_summary_steps': 100, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100}
Tensor("Shape_4:0", shape=(2,), dtype=int32)


TypeError: unhashable type: 'Dimension'